In [1]:
import pandas as pd
import numpy as np
import re
import itertools
import collections
import pickle
from statistics import mean, stdev
from scipy import stats, spatial

In [2]:
# Information for NYUv2
data = 'NYUv2'
iters = 20000
tasks = ('segment_semantic','normal','depth_zbuffer') # task 0, 1, 2
metrics = {'segment_semantic': ['mIoU', 'Pixel Acc'],
           'normal': ['Angle Mean', 'Angle Median', 'Angle 11.25', 'Angle 22.5', 'Angle 30'],
           'depth_zbuffer': ['abs_err','rel_err','sigma_1.25','sigma_1.25^2','sigma_1.25^3']}
metrics_prop = {'mIoU': False, 'Pixel Acc': False, 
                'Angle Mean': True, 'Angle Median': True, 'Angle 11.25': False, 'Angle 22.5': False, 'Angle 30': False,
                'abs_err': True,'rel_err': True,'sigma_1.25': False,'sigma_1.25^2': False,'sigma_1.25^3': False} # True: the lower the better
reduce_var_length = {'train_loss': 10, 'val_loss': 3, 'val_acc': 3}

In [20]:
date = '1129'
model = 'resnet'
B = 5 # 17 - fined, 5 - coarse
coarse = True

In [7]:
date = '0115'
model = 'mobilenet'
B = 6 # 9 - coarse v1, 6 - coarse v2
coarse = True

In [8]:
def confirm_complete(lines, iters):
    rev_lines = lines[::-1]
    for line in rev_lines:
        if 'Iter' in line:
            it = int(line.split(' ')[1])
            break
    if it != iters:
        return False
    else:
        return True

def extract_loss_results(lines, tasks, reduce_var_length):
    # Function: Extract train loss / val loss 
    train_loss = {}
    val_loss = {}
    for task in tasks:
        train_loss[task] = collections.deque(reduce_var_length['train_loss']*[0], reduce_var_length['train_loss'])
        val_loss[task] = collections.deque(reduce_var_length['val_loss']*[0], reduce_var_length['val_loss'])
    
    for line in lines:
        for task in tasks:
            if task[:4] in line and 'Train Loss' in line:
                loss = float(line.split(': ')[1])
                train_loss[task].append(loss)
            elif task[:4] in line and 'Val Loss' in line:
                loss = float(line.split(': ')[1])
                val_loss[task].append(loss)
                
    avg_loss = {'train_loss': [], 'val_loss': []}
    std_loss = {'train_loss': [], 'val_loss': []}
    for task in tasks:
        avg_loss['train_loss'].append(mean(train_loss[task]))
        avg_loss['val_loss'].append(mean(val_loss[task]))
        
        std_loss['train_loss'].append(stdev(train_loss[task]))
        std_loss['val_loss'].append(stdev(val_loss[task]))
    return avg_loss, std_loss

def extract_metric_results(lines, tasks, metrics, reduce_var_length):
    # Function: Extract val metrics
    metric_queue = {}
    for task in tasks:
        for metric in metrics[task]:
            metric_queue[metric] = collections.deque(reduce_var_length['val_acc']*[0], reduce_var_length['val_acc'])
    
    for line in lines:
        for task in tasks:
            for metric in metrics[task]:
                if metric in line:
                    value = float(re.findall("\d+\.\d+", line.split(metric)[1])[0])
                    metric_queue[metric].append(value)
                
    avg_metric = {}
    for task in tasks:
        for metric in metrics[task]:
            avg_metric[metric] = (mean(metric_queue[metric]))
    return avg_metric

def rel_perf(results, lower=True):
    # Function: Compute rel. perf.
    if lower:
        return (results[0,:] - results)/results[0,:]*100
    else:
        return (results - results[0,:])/results[0,:]*100

# For 2-Task Loss

In [9]:
loss_df = {'train_loss': pd.DataFrame(), 'val_loss': pd.DataFrame()}

# For all two tasks
for two_task in set(itertools.combinations(tasks, 2)):
    print(two_task)
    
    # For all branching points
    loss_results = {'train_loss': [], 'val_loss': []}
    loss_std  = {'train_loss': [], 'val_loss': []}
    block = B
    for i in range(block+1):
        log = '_'.join(two_task) + '_b' + str(i) + '.stdout'
        if coarse:
            log = '2task_coarse_'+data+'_'+model+'/'+log
        else:
            log = '2task_fined_'+data+'_'+model+'/'+log
        # Read in content
        with open('./log/'+log) as f:
            lines = f.readlines()
            lines = [line.rstrip() for line in lines]
            if not confirm_complete(lines,iters):
                print(str(i) + ' not complete')
                continue
            avg_loss, std_loss = extract_loss_results(lines, two_task, reduce_var_length)
            for mode in loss_results:
                loss_results[mode].append(avg_loss[mode])
                loss_std[mode].append(std_loss[mode])
    
    # Compute loss std 
    # 1. avg of std of loss in one 2-task model
    # 2. std of loss in multiple 2-task models with different b:
    for mode in loss_results:
        print(mode)
        single_temp = np.array(loss_std[mode])
        multiple_temp = np.array(loss_results[mode])
        for i in range(single_temp.shape[1]):
            print('task '+str(i))
            print('single '+str(mean(single_temp[:,i])))
            print('multiple '+str(stdev(multiple_temp[:,i])))
        print('='*80)
           
    # Add results to dataframe
    task0_idx = tasks.index(two_task[0])
    task1_idx = tasks.index(two_task[1])
    col_name = '(' + str(task0_idx) + ', '+ str(task1_idx) + ')' + '-'
    for mode in loss_df:
        rel_mode_loss = rel_perf(np.array(loss_results[mode]))
        for idx in range(2):
            loss_df[mode][col_name+str(idx)] = rel_mode_loss[:,idx]

('segment_semantic', 'normal')
train_loss
task 0
single 0.01249302902804501
multiple 0.02677682831189021
task 1
single 0.0007691358238131854
multiple 0.0009873696819777727
val_loss
task 0
single 0.01149084769583841
multiple 0.01330411660045557
task 1
single 0.0009567405382894348
multiple 0.0014403574659193551
('segment_semantic', 'depth_zbuffer')
train_loss
task 0
single 0.012872436834668597
multiple 0.019074019590357285
task 1
single 0.0080218243695754
multiple 0.08187166477685263
val_loss
task 0
single 0.013864718082114343
multiple 0.015062251073707645
task 1
single 0.016702541815673225
multiple 0.049194100820683324
('normal', 'depth_zbuffer')
train_loss
task 0
single 0.0007576104586850229
multiple 0.0052310004354382865
task 1
single 0.010555536932761986
multiple 0.0437425597990993
val_loss
task 0
single 0.00037900599330725326
multiple 0.004815100988646469
task 1
single 0.014349063771378612
multiple 0.03308221501981516


In [21]:
# Save to Excel
for mode in loss_df:
    loss_df[mode].to_excel("./2task/"+ data + "_2task_metrics_" + model + "_" + date + "_" + mode + ".xlsx", index_label='branch')  

In [22]:
loss_df

{'train_loss':    (0, 1)-0  (0, 1)-1  (0, 2)-0   (0, 2)-1   (1, 2)-0   (1, 2)-1
 0  0.000000  0.000000  0.000000   0.000000   0.000000   0.000000
 1 -9.487279 -0.088543  2.159702   3.396303 -23.115468  -9.666065
 2 -4.097883  1.770852  1.606371  11.226751 -27.734205 -16.396805
 3  1.607108  4.515672  1.542481  17.528132 -31.111111  -3.354657
 4 -2.331763  2.532318 -0.589839  26.020349 -31.721133   4.967708
 5 -0.803554 -0.832300 -0.192810  30.174625 -35.381264   4.308102
 6 -0.871528  0.194794 -3.078117  28.138301 -38.387800   0.029078,
 'val_loss':    (0, 1)-0  (0, 1)-1  (0, 2)-0   (0, 2)-1   (1, 2)-0  (1, 2)-1
 0  0.000000  0.000000  0.000000   0.000000   0.000000  0.000000
 1 -0.080954  0.052165  1.567627   1.018140 -31.550069 -2.048204
 2  1.237750 -2.556077  1.176766   8.643073 -29.218107 -7.537573
 3 -1.361312 -3.338550  2.267834   8.554153 -31.207133  0.109971
 4  0.419685 -1.095462  0.518362  12.960164 -28.257888  7.707111
 5 -0.187473  2.973396 -0.599879  13.493687 -28.189300 

# For 2-Task Metrics

In [10]:
baselines_2task = {'mIoU': [], 'Pixel Acc': [], 
            'Angle Mean': [], 'Angle Median': [], 'Angle 11.25': [], 'Angle 22.5': [], 'Angle 30': [],
            'abs_err': [],'rel_err': [],'sigma_1.25': [],'sigma_1.25^2': [],'sigma_1.25^3': []}

metric_df = pd.DataFrame()

# For all two tasks
for two_task in set(itertools.combinations(tasks, 2)):
    print(two_task)

    metric_results = {}
    for task in two_task:
        for metric in metrics[task]:
            metric_results[metric] = []
            
    # For all branching points, get metrics results
    block = B
    for i in range(block+1):
        log = '_'.join(two_task) + '_b' + str(i) + '.stdout'
        if coarse:
            log = '2task_coarse_'+data+'_'+model+'/'+log
        else:
            log = '2task_fined_'+data+'_'+model+'/'+log
        # Read in content
        with open('./log/'+log) as f:
            lines = f.readlines()
            lines = [line.rstrip() for line in lines]
            if not confirm_complete(lines,iters):
                print(str(i) + ' not complete')
                continue
            avg_metric = extract_metric_results(lines, two_task, metrics, reduce_var_length)
            for task in two_task:
                for metric in metrics[task]:
                    metric_results[metric].append(avg_metric[metric])
    
    # Take down baselines
    for task in two_task:
        for metric in metrics[task]:
            baselines_2task[metric].append(metric_results[metric][0])
    
    # Compute relative performance for each task
    task_rel_perf = {}
    for task in two_task:
        temp = np.zeros((block + 1, 1))
        idx = 0
        for metric in metrics[task]:
            idx += 1
            temp += rel_perf(np.expand_dims(np.array(metric_results[metric]), axis=1), metrics_prop[metric])
        task_rel_perf[task] = temp/idx
        
    # Add results to dataframe
    task0_idx = tasks.index(two_task[0])
    task1_idx = tasks.index(two_task[1])
    col_name = '(' + str(task0_idx) + ', '+ str(task1_idx) + ')' + '-'
    idx = 0
    for task in two_task:
        metric_df[col_name+str(idx)] = np.squeeze(task_rel_perf[task])
        idx += 1

('segment_semantic', 'normal')
('segment_semantic', 'depth_zbuffer')
('normal', 'depth_zbuffer')


In [24]:
# Save to Excel
metric_df.to_excel("./2task/" + data + "_2task_metrics_" + model + "_" + date + "_val_acc.xlsx", index_label='branch')  

In [25]:
metric_df

,"(0, 1)-0","(0, 1)-1","(0, 2)-0","(0, 2)-1","(1, 2)-0","(1, 2)-1"
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-3.756551,0.206613,1.101209,0.762078,-17.952050,-2.634157
2,-3.049282,-1.543906,2.578223,6.642063,-19.258973,-6.995612
3,-3.226608,-0.321529,2.155586,6.293724,-19.137206,-0.629648
4,-2.031691,0.910665,-0.233959,8.740496,-18.621772,3.480578
5,-2.002587,0.273969,-0.564342,8.774514,-19.128467,1.241206
6,-1.960275,2.428915,-1.746122,12.325905,-18.827467,1.628834


# For Layout Results

In [11]:
def rel_perf_baselines(results, baselines, lower=True):
    # Function: Compute rel. perf. for layouts
    if lower:
        return (baselines - results)/baselines*100
    else:
        return (results - baselines)/baselines*100
    
def save_obj(obj, name):
    with open('./ntask/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open('./ntask/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [21]:
# Layout index for T=3, B=17/5, NYUv2
# layout_idx = [383,342,415,344,278,251,231,161,65,484,329,481,268,374,79,55,368] # verify_1014 + verify_1012(xiao)
# layout_idx = [0,55,72,86,103,200,227,239,349,376,485,493] # verify_1102 (coarse)
# layout_idx = [484,492,487,495,379,397,267,486,490,387] # verify_1102 (flops)
# layout_idx = [484,487] # verify_1116 (similar layouts)
# layout_idx = [0,47,43,34,41,30,44,35,36,49,17,28,26,39,38,10,11,48,42,23,9,33,14,21,5,4,2] # verify_1118 (similar layouts, w/ coarse bps)
layout_idx = [0,45,49,37,34] # top-5

# Layout index for T=3, B=9/6, NYUv2
# layout_idx = [11,18,20,25,24,131,137,111,113,106,110,101,115,125,68,42,66,67,96,100,46,61,91,35,72,88,86,77,84] # verify_1214
# layout_idx = [8,12,14,19,16,62,65,59,61,67,58,53,27,28,45,38,1,48,25,3,23,37,32,22] # verify_0105
base_2task = False # True: baseline from 2task baseline, False: baseline from policymtl
base_2layout = False # True: baseline from layout 2 (the first layout idx)

In [22]:
if base_2task:
    baselines = {'train_loss': [0.5333, 0.0603, 0.5743], 'val_loss': [1.609, 0.0636, 0.6785]}
    # baselines from 2task b0 - only has metrics no loss now
    for metric in baselines_2task:
        baselines[metric] = mean(baselines_2task[metric])
else:
    # baselines are from policymtl
    baselines = {'train_loss': [0.5333, 0.0603, 0.5743], 'val_loss': [1.609, 0.0636, 0.6785],
                 'mIoU': 0.265, 'Pixel Acc': 0.582, 
                 'Angle Mean': 17.7, 'Angle Median': 16.3, 'Angle 11.25': 29.4, 'Angle 22.5': 72.3, 'Angle 30': 87.3,
                 'abs_err': 0.62,'rel_err': 0.24,'sigma_1.25': 57.8,'sigma_1.25^2': 85.8,'sigma_1.25^3': 96} 

In [23]:
loss_results = {'train_loss': [], 'val_loss': []}
metric_results = {}
for task in tasks:
    for metric in metrics[task]:
        metric_results[metric] = []
        
# For each layout
for idx in layout_idx:
    log = 'layout_' + str(idx) + '.stdout'
    with open('./log/layout_'+data+'_'+model+'/'+log) as f:
        lines = f.readlines()
        lines = [line.rstrip() for line in lines]
        if not confirm_complete(lines,iters):
                print(str(idx) + ' not complete')
                continue
        avg_loss,_ = extract_loss_results(lines, tasks, reduce_var_length)
        avg_metric = extract_metric_results(lines, tasks, metrics, reduce_var_length)
        for mode in loss_results:
                loss_results[mode].append(avg_loss[mode])
        for task in tasks:
            for metric in metrics[task]:
                metric_results[metric].append(avg_metric[metric])
                
# Compute rel. perf.
for mode in loss_results:
    loss_results[mode] = rel_perf_baselines(np.array(loss_results[mode]), np.array(baselines[mode]))
    
task_rel_perf = {}
for task in tasks:
    temp = np.zeros(len(layout_idx))
    idx = 0
    for metric in metrics[task]:
        idx += 1
        if base_2layout:
            temp += rel_perf_baselines(np.array(metric_results[metric]), metric_results[metric][-1], metrics_prop[metric])
        else:
            temp += rel_perf_baselines(np.array(metric_results[metric]), baselines[metric], metrics_prop[metric])
    task_rel_perf[task] = temp/idx

In [24]:
for key in task_rel_perf:
    print(key)
    for value in task_rel_perf[key]:
        print(value)

segment_semantic
-0.6232574726058531
-3.111370464025586
-4.416077719423376
-0.800860187166347
-3.4741187404093425
normal
6.371800935045249
4.755908172256467
2.2915749860420433
4.91370000988694
0.011035941535078031
depth_zbuffer
7.759144474720285
5.926193747862684
7.083578878365897
2.9817553579413802
6.868340550240836


In [18]:
# Save all results to one dict
real_results = {}
real_results['layout'] = layout_idx

for mode in loss_results:
    real_results[mode] = loss_results[mode]
    
val_acc = None
for task in task_rel_perf:
    if val_acc is None:
        val_acc = np.expand_dims(task_rel_perf[task], axis=1)
    else:
        val_acc = np.concatenate((val_acc, np.expand_dims(task_rel_perf[task], axis=1)), axis=1)
real_results['val_acc'] = val_acc

In [19]:
real_results

{'layout': [11,
  18,
  20,
  25,
  24,
  131,
  137,
  111,
  113,
  106,
  110,
  101,
  115,
  125,
  68,
  42,
  66,
  67,
  96,
  100,
  46,
  61,
  91,
  35,
  72,
  88,
  86,
  77,
  84],
 'train_loss': array([[-80.91693231,   7.89386401,  11.2171339 ],
        [-61.92012001,   8.57379768,  17.09037089],
        [-56.20101256,   4.84245439, -26.64809333],
        [-53.68648041,  10.72968491, -29.10325614],
        [-56.98481155,  10.89552239, -19.71269371],
        [-59.6624789 ,   5.32338308,  14.52550932],
        [-61.37258579,   7.47927032,  15.54588194],
        [-62.44515282,  -1.6252073 ,   9.08236114],
        [-57.73860866,   2.98507463,  13.35713042],
        [-54.46277892,   6.00331675,   1.23280515],
        [-60.65816614,   4.74295191,   1.49225144],
        [-76.49728108,   7.72802653,  11.97806025],
        [-63.19894993,  -0.86235489,  12.65366533],
        [-60.39564973,   8.8225539 ,  15.37523942],
        [-75.08531783,   7.81094527,  12.31586279],
        [-7

In [15]:
# Save to pickle
save_obj(real_results, 'real_results_'+data+'_'+model+'_'+date)

In [119]:
load_obj('real_results_'+data+'_'+model+'_'+date)

{'layout': [0,
  47,
  43,
  34,
  41,
  30,
  44,
  35,
  36,
  49,
  17,
  28,
  26,
  39,
  38,
  10,
  11,
  48,
  42,
  23,
  9,
  33,
  14,
  21,
  5,
  4,
  2],
 'train_loss': array([[44.09713107, 31.06135987, 50.74003134],
        [37.25295331, 17.03150912, 46.20755703],
        [43.63585224, 14.14593698, 50.33257879],
        [32.79579974,  6.41791045, 46.49138081],
        [38.29739359, 14.12935323, 40.55371757],
        [43.82711419,  5.42288557, 27.44384468],
        [43.46709169, 14.24543947, 48.05676476],
        [40.65254078,  9.2039801 , 37.2366359 ],
        [33.59647478, 10.96185738, 45.76527947],
        [42.26701669, 19.75124378, 49.67612746],
        [35.35720983,  3.30016584, 18.22566603],
        [38.72867054, 18.3747927 , 22.98276162],
        [38.51490718, 26.71641791, 19.13285739],
        [27.98987437, 17.87728027, 23.33275292],
        [41.08944309, 13.53233831, 24.52376807],
        [41.43821489,  3.34991708, 38.92913112],
        [36.66604163,  3.5986733 ,